In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torchinfo
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.2 MB/s eta 0:00:00


In [ ]:
from torch import nn
import torch
import torchinfo
from  torchvision.ops.deform_conv import DeformConv2d
import PIL
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torchvision
import torch.optim as optim
import timm
# from swin_transformer_pytorch import SwinTransformer
import torchinfo
import sys
from tqdm import tqdm
import time
import copy
import os
import pandas as pd

In [ ]:
transform = transforms.Compose([
                                transforms.RandomRotation(45),
                                transforms.RandomVerticalFlip(),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomAffine(degrees = 0, translate = (0.2, 0.2)),
                                transforms.ToTensor()
                                ])
test_transform = transforms.Compose([
                                transforms.ToTensor()
                                    ])
device = "cuda" if torch.cuda.is_available() else "cpu"
kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}

train_dataset_path = '/content/drive/MyDrive/CCPS_research/original_dataset/train_images'
valid_dataset_path = '/content/drive/MyDrive/CCPS_research/original_dataset/valid_images'
test_dataset_path = '/content/drive/MyDrive/CCPS_research/original_dataset/test_images'

train_dataset = datasets.ImageFolder(train_dataset_path, transform=transform)
valid_dataset = datasets.ImageFolder(valid_dataset_path, transform=transform)
test_dataset = datasets.ImageFolder(test_dataset_path,transform=test_transform)

train_dataloader =  torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, **kwargs)
valid_dataloader =  torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=True, **kwargs)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, **kwargs)

CLASSES = train_dataset.classes
train_len = len(train_dataset)
valid_len = len(valid_dataset)
test_len = len(test_dataset)

In [ ]:
dataloaders = {
    "train": train_dataloader,
    "val": valid_dataloader
}
dataset_sizes = {
    "train": train_len,
    "val": valid_len
}

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=3, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input,dim=1)
        # print("logpt is:",logpt)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)
        loss = -1 * (1-pt)**self.gamma * logpt
        # print("loss is :",loss)
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [ ]:
class ChannelShuffle(nn.Module):
  def __init__(self,groups):
    super().__init__()
    self.groups = groups

  def forward(self,x):
    """
    Channel shuffle operation from 'ShuffleNet: An Extremely Efficient Convolutional Neural Network for Mobile Devices,'
    https://arxiv.org/abs/1707.01083.

    Parameters:
    ----------
    x : Tensor
        Input tensor.
    groups : int
        Number of groups.

    Returns
    -------
    Tensor
        Resulted tensor.
    """
    batch, channels, height, width = x.size()
    # assert (channels % groups == 0)
    channels_per_group = channels // self.groups
    x = x.view(batch, self.groups, channels_per_group, height, width)
    # print(x.shape)
    x = torch.transpose(x, 1, 2).contiguous()
    # print(x.shape)
    x = x.view(batch, channels, height, width)
    # print(x.shape)
    return x

In [ ]:
class GSDWR(nn.Module):
  def __init__(self,in_channels,out_channels):
    super().__init__()
    self.groupconv = nn.Conv2d(in_channels = in_channels,out_channels = in_channels,kernel_size = 1,stride=1,groups = 4)
    self.act1 = nn.ReLU()
    self.drop1 = nn.Dropout(0.3)
    self.shuffle = ChannelShuffle(2)

    self.depthwise = nn.Conv2d(in_channels=in_channels,out_channels = out_channels, kernel_size = 1,stride=1, groups = 2)
    self.pointwise = nn.Conv2d(in_channels=out_channels,out_channels = out_channels, kernel_size = 1)

  def forward(self,x):
    inpt = x
    x = self.act1(self.groupconv(x))
    x = self.drop1(x)
    x = self.shuffle(x)
    x = torch.add(x,inpt)
    x = self.depthwise(x)
    x = self.pointwise(x)
    return x

In [ ]:
class DeformableConvBlock(nn.Module):
  def __init__(self,in_channels,out_channels,kernel_size):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=in_channels,out_channels = in_channels, kernel_size=1)
    self.act1 = nn.ReLU()
    self.deformconv = DeformConv2d(in_channels = in_channels, out_channels =in_channels, kernel_size=1,stride = 1)
    self.act2 = nn.ReLU()
    self.depthwise = nn.Conv2d(in_channels=in_channels,out_channels = in_channels, kernel_size = kernel_size, stride=kernel_size, groups = 2)
    self.pointwise = nn.Conv2d(in_channels=in_channels,out_channels = out_channels, kernel_size = 1)

  def forward(self,x):
    # print(x.shape)
    offset = x
    x = self.act1(self.conv1(x))
    x = self.act2(self.deformconv(x,offset))
    x = self.depthwise(x)
    x = self.pointwise(x)
    return x

In [ ]:
class GDBlock(nn.Module):
  def __init__(self,in_channels,factor,out_channels, kernel_size):
    super().__init__()
    self.gsdwr = GSDWR(in_channels,in_channels+4*factor)
    self.deform = DeformableConvBlock(in_channels+4*factor,out_channels,kernel_size)
    self.batchnorm1 = nn.BatchNorm2d(in_channels+4*factor)
    self.batchnorm2 = nn.BatchNorm2d(out_channels)

  def forward(self,x):
    x = self.gsdwr(x)
    x = self.batchnorm1(x)
    x = self.deform(x)
    return x

In [ ]:
class LinearLayer(nn.Module):
  def __init__(self,input):
    super().__init__()
    self.flat = nn.Flatten()
    self.linear1 = nn.Linear(input, 512)
    self.act1 = nn.ReLU()
    self.drop = nn.Dropout(0.3)
    self.linear2 = nn.Linear(512, 23)

  def forward(self,x):
    x = self.flat(x)
    x = self.act1(self.linear1(x))
    x = self.drop(x)
    x = self.linear2(x)
    return x

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels = 3,out_channels = 8, kernel_size=2,stride=2)
    self.gdblock1 = GDBlock(in_channels = 8,factor = 1,out_channels = 16,kernel_size = 1)
    self.batchnorm1 = nn.BatchNorm2d(16)
    self.gdblock2 = GDBlock(in_channels = 16,factor=1,out_channels = 24,kernel_size = 2)
    self.batchnorm2 = nn.BatchNorm2d(24)
    self.gdblock3 = GDBlock(in_channels = 24,factor = 2,out_channels = 40,kernel_size = 2)
    self.batchnorm3 = nn.BatchNorm2d(40)
    self.gdblock4 = GDBlock(in_channels = 40,factor = 5,out_channels = 80,kernel_size = 2)
    self.batchnorm4 = nn.BatchNorm2d(80)
    self.gdblock5 = GDBlock(in_channels = 80,factor = 4,out_channels = 112,kernel_size = 1)
    self.batchnorm5 = nn.BatchNorm2d(112)
    self.gdblock6 = GDBlock(in_channels = 112,factor = 5,out_channels = 192,kernel_size = 2)
    self.linear = LinearLayer(192*7*7)

  def forward(self,x):
    x = self.conv1(x)
    x = self.gdblock1(x)
    x = self.batchnorm1(x)
    x = self.gdblock2(x)
    x = self.batchnorm2(x)
    x = self.gdblock3(x)
    x = self.batchnorm3(x)
    x = self.gdblock4(x)
    x = self.batchnorm4(x)
    x = self.gdblock5(x)
    x = self.batchnorm5(x)
    x = self.gdblock6(x)
    x = self.linear(x)
    return x

In [ ]:
model = Model()
torchinfo.summary(model, (32, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [32, 23]                  --
├─Conv2d: 1-1                            [32, 8, 112, 112]         104
├─GDBlock: 1-2                           [32, 16, 112, 112]        32
│    └─GSDWR: 2-1                        [32, 12, 112, 112]        --
│    │    └─Conv2d: 3-1                  [32, 8, 112, 112]         24
│    │    └─ReLU: 3-2                    [32, 8, 112, 112]         --
│    │    └─Dropout: 3-3                 [32, 8, 112, 112]         --
│    │    └─ChannelShuffle: 3-4          [32, 8, 112, 112]         --
│    │    └─Conv2d: 3-5                  [32, 12, 112, 112]        60
│    │    └─Conv2d: 3-6                  [32, 12, 112, 112]        156
│    └─BatchNorm2d: 2-2                  [32, 12, 112, 112]        24
│    └─DeformableConvBlock: 2-3          [32, 16, 112, 112]        --
│    │    └─Conv2d: 3-7                  [32, 12, 112, 112]        156
│    │    └─

In [ ]:
criterion = FocalLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
# criterion = FocalLoss(alpha =class_weights,gamma=2)
criterion = criterion.to(device)
exp_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=3, threshold=0.01, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-05)
os.chdir("/content/drive/MyDrive/CCPS_research/logs")

In [ ]:
# checkpoint = torch.load(f"checkpoints_cnn_1/model70.pt")
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']
# print(loss)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, path, train_log, valid_log, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    df_train = pd.DataFrame(columns = ['epoch', 'train_loss', 'train_acc'])
    df_val = pd.DataFrame(columns = ['epoch','val_loss','val_acc'])
    for epoch in range(1,num_epochs+1):
        print(f'Epoch {epoch}/{num_epochs}')
        print("-"*10)


        if not os.path.isdir(path):
          os.mkdir(path)
        # else:
        #   checkpoint = torch.load(f"checkpoints/model{epoch-1}.pt")
        #   model.load_state_dict(checkpoint['model_state_dict'])
        #   optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
          # epoch = checkpoint['epoch']
          # loss = checkpoint['loss']

        for phase in ['train', 'val']: # We do training and validation phase per epoch
            if phase == 'train':
                model.train() # model to training mode
            else:
                model.eval() # model to evaluate

            running_loss = 0.0
            running_corrects = 0.0

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'): # no autograd makes validation go faster
                    outputs = model(inputs)
                    # print("outputs are: ",outputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    # print("\nEntering loss")
                    loss = criterion(outputs, labels)
                    # print("loss obtained: ",loss)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc =  running_corrects.double() / dataset_sizes[phase]

            if phase == 'train':
                scheduler.step(epoch_loss) # step at end of epoch

            # print(type(epoch))
            # print(type(epoch_loss))
            # print(type(epoch_acc))
            if phase=='train':
              # print("Done")
              df_new_row = pd.DataFrame({ 'epoch': [epoch], 'train_loss': [epoch_loss],'train_acc':[epoch_acc.cpu()] })
              # print("done2")
              df_train = pd.concat([df_train, df_new_row])
              df_train.to_csv(f'{path}/{train_log}.csv')
            elif phase=='val':
              df_new_row = pd.DataFrame({ 'epoch': [epoch], 'val_loss': [epoch_loss],'val_acc':[epoch_acc.cpu()] })
              df_val = pd.concat([df_val, df_new_row])
              df_val.to_csv(f'{path}/{valid_log}.csv')
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))

            ## save torch model for checkpoints
            if epoch%5==0:
              torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': epoch_loss,
              }, f"{path}/model{epoch}.pt")

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
        print()

    time_elapsed = time.time() - since # slight error
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, "checkpoints_cnn_extended","train_cnn","val_cnn",num_epochs=100)

Epoch 1/100
----------


100%|██████████| 214/214 [00:37<00:00,  5.66it/s]


train Loss: 1.2812 Acc: 0.3556


100%|██████████| 54/54 [00:07<00:00,  7.39it/s]


val Loss: 1.5170 Acc: 0.2907

Epoch 2/100
----------


...

val Loss: 0.3129 Acc: 0.7450

Epoch 100/100
----------


100%|██████████| 214/214 [00:30<00:00,  6.99it/s]


train Loss: 0.2901 Acc: 0.7205


100%|██████████| 54/54 [00:07<00:00,  6.91it/s]


val Loss: 0.3052 Acc: 0.7351

Training complete in 64m 4s
Best Val Acc: 0.7462


In [ ]:
checkpoint = torch.load(f"checkpoints_cnn_extended/model100.pt",map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print(loss)

0.30521407576686194


In [ ]:
predictions = []
label = []
for inputs, labels in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()

    with torch.set_grad_enabled(True): # no autograd makes validation go faster
        outputs = model(inputs)
        # print("outputs are: ",outputs)
        _, preds = torch.max(outputs, 1) # used for accuracy
        for i in preds:
          predictions.append(i)
        for i in labels:
          label.append(i)

100%|██████████| 67/67 [11:27<00:00, 10.26s/it]


In [ ]:
predictions_cpu = [int(i.cpu().numpy()) for i in predictions]
label_cpu = [int(i.cpu().numpy()) for i in label]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(label_cpu, predictions_cpu,zero_division = 0))
# sum(p.numel() for p in x.parameters() if p.requires_grad)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.00      0.00      0.00        11
           2       0.69      0.84      0.76       129
           3       0.75      0.88      0.81       230
           4       0.85      0.81      0.83       202
           5       0.57      0.57      0.57       200
           6       0.61      0.61      0.61       198
           7       0.28      0.14      0.18        81
           8       0.33      0.58      0.42        52
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         2
          11       0.25      0.19      0.22        26
          12       0.64      0.81      0.71       206
          13       0.85      0.79      0.82       200
          14       0.53      0.32      0.40        78
          15       0.80      0.76      0.78       153
          16       0.00      0.00      0.00         7
          17       0.25    

In [ ]:
# conv_layer= nn.Conv2d(in_channels=10, out_channels=32, kernel_size=3)
# sum(p.numel() for p in conv_layer.parameters() if p.requires_grad)